<img src="./images/05-tool.png" width=700px>

We have seen that the LLM needs a complete picture of what it needs to do.

We have been adding extra data to the prompt and then running a new query.

Sometimes we need extra information that we can't provide at the start of the prompt. We can use a Tool to get this information and the LLM will be able to select the correct tool and run it.

We saw in ROUTER how we can get the LLM to select the most appropriate report or in this case tool.

When we say run it, the LLM passes 'us' information on the function that needs to be run and the arguments to be used. 

The LLM can also extract data from the prompt.

We then run the function and send back the output to the LLM by appending to the list of messages.


In the cases where where an Agent calls a tool, the tool is called on our box.

Where are the functions run? From the OpenAI website, we can see at the arrow that tools are executed on our 'box'.

<img style="width:1100px;" src="./images/where-tools-are-executed.png">


Like routing, we can also ask the LLM to select the most appropriate tool to use.

In this example, we see how we can do Tool Selection and Data Extraction. The Agent determines what it will do next and also extracts data from the
LLM response as arguments for the next step. (In 20_planning we will see Tool Calling that use defined functions in our code but for this example,
we will do this manually).

In essence, the LLM decides on the tool/function it wants to use and send back the function name and arguments for us to run on our own box,
get the result and then add this to the messages in the next LLM request.


In [29]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import requests

In [30]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:14]}...")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-TVyeca...


In [31]:
client = OpenAI()
MODEL = "gpt-4o-mini"

In [32]:
# original propmt
system_message = """
You are an assistant that is very good at determining what tool to use to solve queries.
"""

ai_programming = """

# TOOLS

You have two tools:

1. A calculator tool that can perform basic arithmetic operations, such as addition, subtraction, multiplication, and division. If this tool is used then respond in JSON FORMAT.

## Example JSON format:

{"tool": "calculator", "next": "do_calculation", "arguments": {"num1": 5, "num2": 5, "operation": "addition"}} 

2. A jokes tool that can provide a light-hearted joke for the audience reqested. If this tool is used then respond in JSON FORMAT.

## Example JSON format:

{"tool": "joke", "next": "do_joke", "audience": "Pythonistas" }} 

Thank you for your being accurate and complete with this query.
"""
system_message += ai_programming

In [33]:
user_prompt = "What is 102 plus 3?"
# user_prompt = "Tell me a joke when I am doing stand up at a Python Conference"
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt},
]
response = client.chat.completions.create(model=MODEL, messages=prompts)
print(response.choices[0].message.content)
output = response.choices[0].message.content.replace("\n", "")


{"tool": "calculator", "next": "do_calculation", "arguments": {"num1": 102, "num2": 3, "operation": "addition"}}


In [34]:
# As specified in the AI Programming section of the prompt, we get back a JSON response/dictionary with what
# the next step should be.

print(output)

{"tool": "calculator", "next": "do_calculation", "arguments": {"num1": 102, "num2": 3, "operation": "addition"}}


Having got the function and arguments, we extract these from the response and run it.


In [35]:
json_output = output.strip()
json_output = json.loads(json_output)
print(json_output)
# use do_next as 'next' is a reserved word
do_next = json_output["next"]

{'tool': 'calculator', 'next': 'do_calculation', 'arguments': {'num1': 102, 'num2': 3, 'operation': 'addition'}}


Now we use our day to day Python to run some code based on the next step, _very much like calling a Weather API and then writing code based on the result._

There are other ways we might code this but this will suffice for demonstration purposes.

In this demo, we use the calculator tool or joke tool to give a response. The tool is some code rather than a defined functuion, we will see this in the PLANNING demo `20_planning`, we will use the tools to call functions in our code which we then feedback to the LLM to carry out the next stage of the plan.


In [36]:
# this could be encapsulated into a function
if do_next == "do_joke":
    get_random_joke_internet = requests.get(
        "https://official-joke-api.appspot.com/random_joke"
    )
    print(get_random_joke_internet.json())

In [37]:
# this could be encapsulated into a function
# the main point is we have extracted the tool and data from the response
if do_next == "do_calculation":
    tool = json_output["tool"]
    num1 = json_output["arguments"]["num1"]
    num2 = json_output["arguments"]["num2"]
    operation = json_output["arguments"]["operation"]
    if operation == "addition":
        answer = num1 + num2
    elif operation == "subtraction":
        answer = num1 - num2
    elif operation == "multiplication":
        answer = num1 * num2
    elif operation == "division":
        answer = num1 / num2
    print(f"{num1} {operation} {num2} = {answer}")

102 addition 3 = 105
